In [1]:
#!pip install shap
#!pip install rfpimp

In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import shap 
#import rfpimp

import warnings

import time

from scipy.stats import logistic

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [3]:
#https://github.com/parrt/random-forest-importances/blob/master/src/rfpimp.py
def oob_regression_r2_score(rf, X_train, y_train):
    """
    Compute out-of-bag (OOB) R^2 for a scikit-learn random forest
    regressor. We learned the guts of scikit's RF from the BSD licensed
    code:
    https://github.com/scikit-learn/scikit-learn/blob/a24c8b46/sklearn/ensemble/forest.py#L702
    """
    X = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    y = y_train.values if isinstance(y_train, pd.Series) else y_train

    n_samples = len(X)
    predictions = np.zeros(n_samples)
    n_predictions = np.zeros(n_samples)
    for tree in rf.estimators_:
        unsampled_indices = generate_unsampled_indices(tree.random_state, n_samples)
        tree_preds = tree.predict(X[unsampled_indices, :])
        predictions[unsampled_indices] += tree_preds
        n_predictions[unsampled_indices] += 1

    if (n_predictions == 0).any():
        warnings.warn("Too few trees; some variables do not have OOB scores.")
        n_predictions[n_predictions == 0] = 1

    predictions /= n_predictions

    oob_score = r2_score(y, predictions)
    return oob_score

#http://bakfu.github.io/doc/_modules/sklearn/ensemble/forest.html
from sklearn.utils import check_random_state #, check_array, compute_sample_weight
#from sklearn.utils.fixes import bincount

def generate_sample_indices(random_state, n_samples):
    """Private function used to _parallel_build_trees function."""
    random_instance = check_random_state(random_state)
    sample_indices = random_instance.randint(0, n_samples, n_samples)

    return sample_indices

def generate_unsampled_indices(random_state, n_samples):
    """Private function used to forest._set_oob_score fuction."""
    sample_indices = generate_sample_indices(random_state, n_samples)
    sample_counts = np.bincount(sample_indices, minlength=n_samples)
    unsampled_mask = sample_counts == 0
    indices_range = np.arange(n_samples)
    unsampled_indices = indices_range[unsampled_mask]

    return unsampled_indices

def shap_values_oob(X_train, rf):
    n_samples, p = X_train.shape
    k=0
    shap_oob = np.zeros((n_samples, p, rf.n_estimators))
    shap_inbag = np.zeros((n_samples, p, rf.n_estimators))
    for tree in rf.estimators_:
      tree_preds = tree.predict(X_train)
      unsampled_indices = generate_unsampled_indices(tree.random_state, n_samples)
      sampled_indices = generate_sample_indices(tree.random_state, n_samples)
      explainer = shap.TreeExplainer(tree)
      shap_oob[unsampled_indices,:,k] = explainer.shap_values(X_train.iloc[unsampled_indices,:])
      shap_inbag[sampled_indices,:,k] = explainer.shap_values(X_train.iloc[sampled_indices,:])
      #print(k)
      k+=1
    
    shap_oob_avg = np.sum(shap_oob, axis=2) 
    shap_inbag_avg = np.sum(shap_inbag, axis=2)
    globalSHAPImp_oob =np.sum(np.abs(shap_oob_avg), axis=0)
    globalSHAPImp_inbag = np.sum(np.abs(shap_inbag_avg), axis=0)
    return shap_oob,shap_inbag,shap_oob_avg,shap_inbag_avg,globalSHAPImp_oob,globalSHAPImp_inbag
    

In [67]:
def NoisyFeatureIdentification(N=1000, # number of rows in data
                        p=50, # number of features
                        #nCores = M, # number of cores to use; set to 1 on Windows!
                        relevance = 0.15, # signal srength (0 for NULL)
                        inoutbag=False,
                        ntree = 100, #number of trees in forest
                        #correctBias = c(inbag=TRUE,outbag=TRUE),
                        verbose=0,
                        n_features = 2):
  
  #VERY dangerous to put this into the function
  #random.seed(123)

  shap_avs = np.array([0,0,0,0,0]) # Initializes the first array
  ft_importances = np.array([0,0,0,0,0]) # Initializes the first array
  shap_vals = np.array([0,0,0,0,0]) # Initializes the first array
  allDFs = [] # List of DFs that will be filled


  allXs = []

  for i in range(p):
    allXs.append([np.random.randint(0, i+2, N)])#i+2
  
  rlvFtrs = np.array([0] * p)
  # rlvFtrs[np.random.randint(1, 11, 5)] = 1
  rlvFtrs[np.random.choice(range(0, 10), 5, replace=False)] = 1
  position = np.where(rlvFtrs == 1)[0]

  Xrlv = np.array([0]*N) 

  for k in (np.where(rlvFtrs == 1)[0]):
    Xrlv = Xrlv + allXs[k]/(k+1)

  y = np.array([]) 

  for i in range(N):
    y = np.append(y, np.random.binomial(n = 1, p = logistic.cdf(2*Xrlv[0][i]/5 - 1), size = 1))

  x_train = pd.DataFrame(np.concatenate(allXs).T.reshape(N, p, order='F')) 


  rf = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=ntree,max_features=n_features) 
  rf.fit(x_train, y)
  feature_importances = rf.feature_importances_
  #imp = permutation_importances(rf, X_train, y,oob_regression_r2_score)
  # print(feature_importances)
  
  warnings.filterwarnings('ignore')
  if (inoutbag):
    shap_oob,shap_inbag,shap_oob_avg,shap_inbag_avg,globalSHAPImp_oob,globalSHAPImp_inbag = shap_values_oob(x_train, rf)
    return(shap_oob,shap_inbag,shap_oob_avg,shap_inbag_avg,globalSHAPImp_oob,globalSHAPImp_inbag, rlvFtrs,y)
  else:
    shap_values = shap.TreeExplainer(rf, feature_perturbation="tree_path_dependent" ).shap_values(x_train)
    #shap_values = shap.TreeExplainer(rf).shap_values(x_train)
    shap_averages = np.sum(np.absolute(shap_values), axis=0)
    return(shap_values, shap_averages, feature_importances, x_train, rlvFtrs)
    

  '''
  shap_vals = np.vstack((shap_vals, shap_values))
  shap_avs = np.vstack((shap_avs, shap_averages))
  ft_importances = np.vstack((ft_importances, feature_importances))
  allDFs.append(x_train)
  
  shap_vals = np.delete(shap_vals, (0), axis=0) # Deletes the initialization
  shap_avs = np.delete(shap_avs, (0), axis=0) # Deletes the initialization
  ft_importances = np.delete(ft_importances, (0), axis=0) # Deletes the initialization
  '''


In [ ]:
start_time = time.time()
random.seed(123)
N=1000;p0=50

shap_oob, shap_inbag, shap_oob_all, shap_inbag_all, globalSHAPImp_oob, globalSHAPImp_inbag, rlvFtrs_all, y_train_all = NoisyFeatureIdentification(N=N, p=p0, inoutbag=True)
#shap_oob_all=globalSHAPImp_oob
#shap_inbag_all=globalSHAPImp_inbag
nSims = 50

for i in range(nSims-1):

    shap_oob, shap_inbag, shap_oob_avg, shap_inbag_avg, globalSHAPImp_oob, globalSHAPImp_inbag, rlvFtrs, y_train = NoisyFeatureIdentification(N=N, p=p0, inoutbag=True)
    print("done with iteration", i)
    
    shap_oob_all = np.vstack((shap_oob_all, shap_oob_avg))
    shap_inbag_all = np.vstack((shap_inbag_all, shap_inbag_avg))
    #shap_oob_all = np.vstack((shap_oob_all, globalSHAPImp_oob))
    #shap_inbag_all = np.vstack((shap_inbag_all, globalSHAPImp_inbag))
    rlvFtrs_all = np.hstack((rlvFtrs_all,rlvFtrs))
    y_train_all = np.hstack((y_train_all,y_train))



print("--- %s seconds ---" % (time.time() - start_time))

done with iteration 0
done with iteration 1
done with iteration 2
done with iteration 3
done with iteration 4
done with iteration 5
done with iteration 6
done with iteration 7
done with iteration 8
done with iteration 9
done with iteration 10
done with iteration 11
done with iteration 12
done with iteration 13
done with iteration 14
done with iteration 15
done with iteration 16
done with iteration 17
done with iteration 18
done with iteration 19
done with iteration 20
done with iteration 21
done with iteration 22
done with iteration 23
done with iteration 24
done with iteration 25
done with iteration 26
done with iteration 27
done with iteration 28
done with iteration 29
done with iteration 30
done with iteration 31
done with iteration 32
done with iteration 33
done with iteration 34
done with iteration 35
done with iteration 36
done with iteration 37
done with iteration 38
done with iteration 39
done with iteration 40
done with iteration 41
done with iteration 42
done with iteration 4

In [69]:
#shap_averages_oob = np.sum(np.absolute(), axis=0) 

#shap_oob, shap_inbag, shap_oob_all, shap_inbag_all, globalSHAPImp_oob, globalSHAPImp_inbag, rlvFtrs_all = NoisyFeatureIdentification(N=100, p=10, inoutbag=True)
#shap_oob, shap_inbag, shap_oob_avg, shap_inbag_avg, globalSHAPImp_oob, globalSHAPImp_inbag, rlvFtrs = NoisyFeatureIdentification(N=100, p=10, inoutbag=True)
rlvFtrs_all.tofile("data/rlvFtrs_all.tsv","\t")
shap_oob_all.tofile("data/shap_oob_all.tsv","\t")
shap_inbag_all.tofile("data/shap_inbag_all.tsv","\t")
y_train_all.tofile("data/y_train_all.tsv","\t")

In [79]:
shap_oob_all_wghted = (shap_oob_all.T*y_train_all).T
shap_inbag_all_wghted = (shap_inbag_all.T*y_train_all).T

print(shap_inbag_all_wghted.shape,shap_oob_all_wghted.shape)

(50000, 50) (50000, 50)


In [80]:
i=-1
shap_oob_all_wghted_avg =  np.sum(np.absolute(shap_oob_all_wghted[np.arange(N*(i+1),N*(i+2)),:]), axis=0) 
shap_inbag_all_wghted_avg =  np.sum(np.absolute(shap_inbag_all_wghted[np.arange(N*(i+1),N*(i+2)),:]), axis=0) 
print(shap_oob_all_wghted_avg.shape, shap_inbag_all_wghted_avg.shape)
np.vstack((shap_oob_all_wghted_avg,shap_inbag_all_wghted_avg)).shape

(50,) (50,)


(2, 50)

In [81]:
#rlvFtrs_all = np.hstack((rlvFtrs_all,rlvFtrs))
i=-1
shap_oob_all_wghted_avg =  np.sum(np.absolute(shap_oob_all_wghted[np.arange(N*(i+1),N*(i+2)),:]), axis=0) 
shap_inbag_all_wghted_avg =  np.sum(np.absolute(shap_inbag_all_wghted[np.arange(N*(i+1),N*(i+2)),:]), axis=0) 
print(shap_oob_all_wghted_avg.shape,shap_inbag_all_wghted_avg.shape)

for i in range(nSims-1):
    tmp = np.sum(np.absolute(shap_oob_all_wghted[np.arange(N*(i+1),N*(i+2)),:]), axis=0)
    shap_oob_all_wghted_avg = np.vstack((shap_oob_all_wghted_avg, tmp ))
    tmp = np.sum(np.absolute(shap_inbag_all_wghted[np.arange(N*(i+1),N*(i+2)),:]), axis=0) 
    shap_inbag_all_wghted_avg = np.vstack((shap_inbag_all_wghted_avg, tmp ))

print(shap_oob_all_wghted_avg.shape,shap_inbag_all_wghted_avg.shape,rlvFtrs_all.shape)
    

(50,) (50,)
(50, 50) (50, 50) (2500,)


In [83]:
rlvFtrs_oneArray = rlvFtrs_all #np.concatenate(rlvFtrs)
shap_oob_avg_oneArray = np.concatenate(shap_oob_all_wghted_avg)#shap_oob_all)
shap_inbag_avg_oneArray = np.concatenate(shap_inbag_all_wghted_avg)#shap_inbag_all)
print(rlvFtrs_oneArray.shape, shap_oob_avg_oneArray.shape, shap_inbag_avg_oneArray.shape)

fpr, tpr, thresholds = metrics.roc_curve(rlvFtrs_oneArray, shap_oob_avg_oneArray)
print("oob", metrics.auc(fpr, tpr))

fpr, tpr, thresholds = metrics.roc_curve(rlvFtrs_oneArray, shap_inbag_avg_oneArray)
print("inbag", metrics.auc(fpr, tpr))

(2500,) (2500,) (2500,)
oob 0.7247822222222221
inbag 0.5603946666666666


In [71]:
#test
print(shap_oob_avg.shape)
print(rlvFtrs_all.shape, shap_oob_all.shape, shap_inbag_all.shape, y_train_all.shape)
#print(rlvFtrs_all.shape, shap_oob_all.shape, shap_inbag_all.shape)

(1000, 50)
(2500,) (50000, 50) (50000, 50) (50000,)


In [73]:
#shap_inbag_avg.append(fourth)
#rlvFtrs.append(seventh)
rlvFtrs_oneArray = rlvFtrs_all #np.concatenate(rlvFtrs)
shap_oob_avg_oneArray = np.concatenate(shap_oob_all_wghted)#shap_oob_all)
shap_inbag_avg_oneArray = np.concatenate(shap_inbag_all_wghted)#shap_inbag_all)
print(rlvFtrs_oneArray.shape, shap_oob_avg_oneArray.shape, shap_inbag_avg_oneArray.shape)

(2500,) (2500000,) (2500000,)


In [66]:
fpr, tpr, thresholds = metrics.roc_curve(rlvFtrs_oneArray, shap_oob_avg_oneArray)
print("oob", metrics.auc(fpr, tpr))

fpr, tpr, thresholds = metrics.roc_curve(rlvFtrs_oneArray, shap_inbag_avg_oneArray)
print("inbag", metrics.auc(fpr, tpr))

oob 0.6949937777777778
inbag 0.5164213333333334


In [9]:
start_time = time.time()
random.seed(123)

shap_vals, shap_avs, ft_importances, x_train, rlvFtrs = ([] for i in range(5))
nSims = 100

for i in range(nSims):

  first, second, third, fourth, fifth = NoisyFeatureIdentification(N=1000, p=50)
  print("done with iteration" % i)
  shap_vals.append(first)
  shap_avs.append(second)
  ft_importances.append(third)
  x_train.append(fourth)
  rlvFtrs.append(fifth)


print("--- %s seconds ---" % (time.time() - start_time))

--- 1120.602492570877 seconds ---


In [ ]:
shap.summary_plot(shap_vals[0])

In [ ]:
plt.boxplot(shap_avs[0]);

In [ ]:
plt.boxplot(ft_importances);

In [10]:
# save with pickle
import pickle
pickle.dump(shap_avs, open("shap_avs.p", "wb"))
pickle.dump(rlvFtrs, open("rlvFtrs.p", "wb"))

In [43]:
# load with pickle
import pickle
shap_avs2 = pickle.load(open("shap_avs.p", "rb"))
rlvFtrs2 = pickle.load(open("rlvFtrs.p", "rb"))

In [60]:
rlvFtrs_oneArray = np.concatenate(rlvFtrs2)
shap_avs_oneArray2 = np.concatenate(shap_avs2)
print(rlvFtrs_oneArray.shape)
print(shap_avs_oneArray2.shape)
len(shap_avs2[0])

(5000,)
(5000,)


50

In [11]:
rlvFtrs_oneArray = np.concatenate(rlvFtrs)
shap_avs_oneArray = np.concatenate(shap_avs)

fpr, tpr, thresholds = metrics.roc_curve(rlvFtrs_oneArray, shap_avs_oneArray)
metrics.auc(fpr, tpr)

0.6576973333333334

In [ ]:
## Test von x_train ##

In [ ]:
allXs = []

for i in range(p):
    allXs.append([np.random.randint(0, i+2, N)])#i+2

pd.DataFrame(np.concatenate(allXs).reshape(N, p, order='F')) 

In [ ]:
allXs2 = []
N = 10
p = 4

for i in range(p):
  allXs2.append([np.arange(1+i*N,1+N+i*N)])


In [ ]:
allXs

In [ ]:
allXs2

In [ ]:
np.concatenate(allXs2).T

In [ ]:
pd.DataFrame(np.concatenate(allXs2).T.reshape(N, p, order = "F"))

In [ ]:
## ##

In [ ]:
#explorations:
allXs = []

for i in range(8):
  allXs.append([np.random.randint(0, i+2, 100)])

y= np.random.binomial(n = 1, p = ==0.5, size = 100)
x_train = pd.DataFrame(np.concatenate(allXs).reshape(100, 8, order='F'))
x_train.shape

rf = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=50,max_features=2) 
rf.fit(x_train, y)

#imp =  rfpimp.importances(rf, x_train, y)

In [ ]:
shap_vals, shap_avs, ft_importances, x_train, rlvFtrs = SimulateData_simple(n_features=10)

In [ ]:
print(ft_importances[np.where(rlvFtrs == 1)[0]])
print(ft_importances[np.where(rlvFtrs == 0)[0]])

In [ ]:
#shap_avs.shape
rlvFtrs
shap.summary_plot(shap_vals)

In [ ]:
#shap_avs.shape  #[np.where(rlvFtrs == 1)[0]]
#shap_vals.shape
#tmp= np.sum(np.absolute(shap_vals), axis=1)
#tmp.shape
shap.summary_plot(shap_vals, plot_type="bar",max_display=50)
np.where(rlvFtrs == 1)